In [1]:
import os

import dotenv
import pandas as pd
import pymongo

In [2]:
dotenv.load_dotenv()
mongodb_uri = os.getenv("MONGODB_URI")
DBclient = pymongo.MongoClient(mongodb_uri)
db = DBclient["ML2-Abschlussprojekt"]

_________

# Drop Collections

In [3]:
def drop_all_collections():
    db["Base_Data"].drop()
    db["Semester_Data"].drop()
    db["Assessment_Data"].drop()

In [4]:
drop_all_collections()

_________

# Base Data

In [5]:
df_base = pd.read_excel("Student-Data.xlsx", "Base_Data").convert_dtypes().replace({pd.NA:None})
df_base.head()

,firstname,lastname,gender,birthday,school_entry,diagnose,compensation,class_teacher,special_education_teacher
0,Bruce,Banner,männlich,2009-02-02,2022-08-01,Schizophrenie,Separater Raum bei Prüfungen,Nick Fury,Miss Marvel
1,Natasha,Romanoff,weiblich,2009-03-03,2022-08-01,None,None,Nick Fury,Miss Marvel


In [6]:
base_data = [
    row.to_dict() for col, row in df_base.iterrows()
]

In [7]:
collection = db["Base_Data"]
result = collection.insert_many(base_data)

# Mapping: Prename to ObjectIDs from MongoDB
student_map = {
    s["firstname"]: r
    for s,r in zip(base_data, result.inserted_ids)
}

In [8]:
student_map

{'Bruce': ObjectId('6663031cb9767114ee3d1048'),
 'Natasha': ObjectId('6663031cb9767114ee3d1049')}

____________

# Semester Data

In [9]:
df_semester = pd.read_excel("Student-Data.xlsx", "Semester_Data").convert_dtypes().replace({pd.NA:None})
df_semester.head()

,firstname,semester_name,grade_level,niveau,last_progress_talk,last_report
0,Bruce,FS24,1. Sekundarstufe,A,FS24,HS23
1,Natasha,FS24,3. Sekundarstufe,B,FS24,HS23
2,Bruce,HS23,6. Klasse,keines,FS23,FS23
3,Natasha,HS23,2. Sekundarstufe,B,FS23,FS23
4,Bruce,FS23,6. Klasse,keines,HS22,HS22


In [10]:
# Create dictionary with all the important information, including ObjectID-reference to the student
semester_data = {
    (row.firstname, row.semester_name): dict(
        student_id=student_map[row.firstname],
        **{
            k:v for k,v in
            row.to_dict().items()
            if k != "firstname"
        },
        final_assessment=None, # will be needed later
        final_text=None, # will be needed later
    )
    for _, row in df_semester.iterrows()
}
list(semester_data.values())[0]

{'student_id': ObjectId('6663031cb9767114ee3d1048'),
 'semester_name': 'FS24',
 'grade_level': '1. Sekundarstufe',
 'niveau': 'A',
 'last_progress_talk': 'FS24',
 'last_report': 'HS23',
 'final_assessment': None,
 'final_text': None}

In [11]:
# Load Entries to MongoDB
collection = db["Semester_Data"]
result = collection.insert_many(semester_data.values())

# Mapping: (student_id, semester) to ObjectIDs of Evaluations
semester_map = {
    k: r
    for k,r in zip(semester_data.keys(), result.inserted_ids)
}

In [12]:
semester_map

{('Bruce', 'FS24'): ObjectId('6663031eb9767114ee3d104a'),
 ('Natasha', 'FS24'): ObjectId('6663031eb9767114ee3d104b'),
 ('Bruce', 'HS23'): ObjectId('6663031eb9767114ee3d104c'),
 ('Natasha', 'HS23'): ObjectId('6663031eb9767114ee3d104d'),
 ('Bruce', 'FS23'): ObjectId('6663031eb9767114ee3d104e'),
 ('Natasha', 'FS23'): ObjectId('6663031eb9767114ee3d104f')}

____________

# Assessments

In [13]:
df_assessment = pd.read_excel(
    "Student-Data.xlsx",
    "Assessments_Data",
    converters=dict(date=pd.to_datetime),
).convert_dtypes().replace({pd.NA:None})
df_assessment.head()

,firstname,semester_name,author,AktivTeilnehmen,AktivTeilnehmenNotizen,LeistungZeigen,LeistungZeigenNotizen,AufmerksamSein,AufmerksamSeinNotizen,SchulinhalteMerken,SchulinhalteMerkenNotizen,SchulinhalteAbrufen,SchulinhalteAbrufenNotizen,date
0,Bruce,HS23,Teacher1,gut,"insbesondere bei Fächer, die ihn interessieren",gut,teilweise könnte er noch etwas mehr Leistung z...,sehr gut,None,gut,None,teilweise,"ab und zu fällt es Bruce schwer, bei Prüfungen...",2023-12-01 09:42:14.695000+00:00
1,Bruce,HS23,Teacher2,sehr gut,Bruce hat viele schulische Interessen. Insbeso...,gut,None,gut,Oft ist er im Unterricht aufmerksam.,teilweise,er vegisst nach den Prüfungen vieles,nicht immer,"manchmal hat er Schwierigkeiten, bei Prüfungen...",2023-12-01 09:43:13.695000+00:00
2,Bruce,HS23,Teacher3,gut,Bruce ist meistens aktiv in der Schule,sehr gut,None,gut,"Es kann vorkommen, dass er etwas verträumt ist.",teilweise,None,nicht immer,None,2023-12-01 09:44:13.695000+00:00
3,Bruce,FS23,Teacher1,gut,None,teilweise,teilweise ist er sehr müde,teilweise,None,nicht immer,"bei Themen, die ihn interessieren, kann er sic...",noch nicht,None,2023-06-01 09:42:13.695000+00:00
4,Bruce,FS23,Teacher2,sehr gut,er meldet sich oft im Unterricht,teilweise,"wenn er nicht müde ist, geht es gut",gut,None,teilweise,None,nicht immer,None,2023-06-01 09:43:14.695000+00:00


In [14]:
# Create dictionary with all the important information, including ObjectID - reference to the student
assessment_data = []
assessment_critera = {
    "AktivTeilnehmen",
    "LeistungZeigen",
    "AufmerksamSein",
    "SchulinhalteMerken",
    "SchulinhalteAbrufen",
}
for _, row in df_assessment.iterrows():
    rowdict = row.to_dict()
    assessment = {
        criterium: dict(
            assessment=rowdict.pop(criterium),
            notes=rowdict.pop(criterium+"Notizen"),
        ) for criterium in assessment_critera
    }
    # Rule: All Assessment-Criteria are in UpperCase and we only want to handle these
    assert not any(k[0].isupper() for k in rowdict)

    rowdict.pop("firstname")
    rowdict.pop("semester_name")
    
    
    assessment_data.append(dict(
        student_id=student_map[row.firstname],
        semester_id=semester_map[row.firstname, row.semester_name],
        allgemeines_lernen=assessment,
        **rowdict,
    ))

assessment_data[0]

{'student_id': ObjectId('6663031cb9767114ee3d1048'),
 'semester_id': ObjectId('6663031eb9767114ee3d104c'),
 'allgemeines_lernen': {'LeistungZeigen': {'assessment': 'gut',
   'notes': 'teilweise könnte er noch etwas mehr Leistung zeigen, wenn er rechtzeit lernen würde'},
  'SchulinhalteAbrufen': {'assessment': 'teilweise',
   'notes': 'ab und zu fällt es Bruce schwer, bei Prüfungen zu zeigen, was er kann.'},
  'AktivTeilnehmen': {'assessment': 'gut',
   'notes': 'insbesondere bei Fächer, die ihn interessieren'},
  'AufmerksamSein': {'assessment': 'sehr gut', 'notes': None},
  'SchulinhalteMerken': {'assessment': 'gut', 'notes': None}},
 'author': 'Teacher1',
 'date': Timestamp('2023-12-01 09:42:14.695000+0000', tz='UTC')}

In [15]:
collection = db["Assessment_Data"]
result = collection.insert_many(assessment_data)

________

# Final Assessment

In [16]:
data = {
    "allgemeines_lernen": {
        "AktivTeilnehmen": {
            "assessment":"sehr gut",
            "notes":"Bruce hat viele schulische Interessen und ist in der Schule meist aktiv, insbesondere in Physik und Chemie ist er sehr engagiert."
        },
        "LeistungZeigen": {
            "assessment":"gut",
            "notes":"teilweise könnte er noch etwas mehr Leistung zeigen, wenn er rechtzeit lernen würde"
        },
        "AufmerksamSein": {
            "assessment":"teilweise",
            "notes":"In der Regel ist Bruce im Unterricht aufmerksam und konzentriert. Es kann jedoch vorkommen, dass er manchmal etwas abgelenkt wirkt."
        },
        "SchulinhalteMerken": {
            "assessment":"teilweise",
            "notes":""
        },
        "SchulinhalteAbrufen": {
            "assessment":"nicht immer",
            "notes":"Gelegentlich fällt es Bruce schwer, sein Wissen bei Prüfungen abzurufen. Manchmal hat er Schwierigkeiten, sein volles Potenzial zu zeigen."
        },
    }
}

In [17]:
semester_id = semester_map.get(('Bruce', 'HS23'))

In [18]:
collection = db["Semester_Data"]
filter = {
    "_id":semester_id,
}
entry = {"$set": {"final_assessment": data}}
collection.update_one(filter, entry, upsert=True)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000445'), 'opTime': {'ts': Timestamp(1717764902, 31), 't': 1093}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1717764902, 33), 'signature': {'hash': b"\xba\x17\xb3\x84\x1e)27\xff\xd0\x81'\x02\xb2\x9f\x8b\x80\xaa.X", 'keyId': 7340684186488930306}}, 'operationTime': Timestamp(1717764902, 31), 'updatedExisting': True}, acknowledged=True)

__________